# `make_rid_to_station_dict.ipynb`

### Author: Anthony Hein

#### Last updated: 10/18/2021

# Overview:

At this point we have a dictionary which matches courses to locations (using latitude and longitude) and metadata on all weather stations in Great Britain that have published their data (where this metadata includes open and close dates of the station as well as the latitue and longitude of the station). Therefore, the next step is to match each race to a weather station, where we will select the weather station that is active during the race and nearest to the course that the race is taking place at.

---

## Setup

In [1]:
from datetime import datetime
import git
import os
from typing import List, Union
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
BASE_DIR = git.Repo(os.getcwd(), search_parent_directories=True).working_dir
BASE_DIR

'/Users/anthonyhein/Desktop/SML310/project'

In [3]:
import sys

sys.path.append(f'{BASE_DIR}/utils/')

from course_and_country_to_location import COURSE_AND_COUNTRY_TO_LOCATION
from rid_to_course_and_country import RID_TO_COURSE_AND_COUNTRY

---

## Load `horses_aticnmg.csv`

In [4]:
horses_aticnmg = pd.read_csv(f"{BASE_DIR}/data/csv/horses_aticnmg.csv", low_memory=False) 
horses_aticnmg.head()

,rid,horseName,age,saddle,decimalPrice,trainerName,jockeyName,position,positionL,dist,outHandicap,RPR,TR,OR,father,mother,gfather,weight
0,267255,Going For Broke,3.0,4.0,0.100000,P C Haslam,Seb Sanders,1,NaN,0.0,0.0,72.0,62.0,62.0,Simply Great,Empty Purse,Pennine Walk,58
1,267255,Pinchincha,3.0,3.0,0.266667,Dave Morris,Tony Clark,2,4,0.0,0.0,66.0,56.0,65.0,Priolo,Western Heights,Shirley Heights,60
2,267255,Skelton Sovereign,3.0,5.0,0.142857,Reg Hollinshead,D Griffiths,3,3,7.0,0.0,55.0,40.0,60.0,Contract Law,Mrs Lucky,Royal Match,55
3,267255,Fast Spin,3.0,6.0,0.380952,David Barron,Tony Culhane,4,7,14.0,0.0,38.0,30.0,59.0,Formidable I,Topwinder,Topsider,57
4,267255,As-Is,3.0,2.0,0.166667,Mark Johnston,J Weaver,5,7,21.0,0.0,29.0,21.0,65.0,Lomond,Capriati I,Diesis,60


In [5]:
horses_aticnmg.shape

(1146202, 18)

---

## Load `races_aticnmg.csv`

In [6]:
races_aticnmg = pd.read_csv(f"{BASE_DIR}/data/csv/races_aticnmg.csv", low_memory=False) 
races_aticnmg.head()

,rid,course,time,date,hurdles,prizes,winningTime,metric,countryCode,ncond,class
0,267255,Southwell (AW),03:40,97/01/01,NaN,"[2752.25, 833.0, 406.5, 193.25]",106.9,1609.0,GB,0,5
1,297570,Southwell (AW),12:35,97/01/01,NaN,"[1944.0, 544.0, 264.0]",91.0,1407.0,GB,0,6
2,334421,Southwell (AW),01:05,97/01/01,NaN,"[2502.0, 702.0, 342.0]",150.7,2212.0,GB,0,6
3,366304,Southwell (AW),03:10,97/01/01,NaN,"[2189.0, 614.0, 299.0]",108.6,1609.0,GB,0,6
4,13063,Southwell (AW),02:40,97/01/01,NaN,"[2726.25, 825.0, 402.5, 191.25]",231.4,3318.5,GB,0,5


In [7]:
races_aticnmg.shape

(123972, 11)

---

## Load `stations_metadata.csv`

Dataset was obtained at [https://archive.ceda.ac.uk/cgi-bin/midas_stations/search_by_name.cgi.py?name=&opendata=y&minyear=&maxyear=](https://archive.ceda.ac.uk/cgi-bin/midas_stations/search_by_name.cgi.py?name=&opendata=y&minyear=&maxyear=). This was accomplished by copy-pasting the HTML table into a file, replacing the tabs with commas, and changing instances of `Current` to `2022-01-01` (note that the horse race data we are using does not go to 2022 so this suffices for our purposes). These measures had to be taken because when I tried downloading it or pasting it into Excel, it kept formatting the dates in an undesirable format that made them difficult to parse.

In [8]:
stations_metadata = pd.read_csv(f"{BASE_DIR}/data/csv/stations_metadata.csv", low_memory=False) 
stations_metadata.head()

,src_id,Station name,Area,Area type,Station start date,Station end date,Latitude,Longitude,Postcode,Open datasets available
0,13638,GLASGOW: OLD OBSERVATORY,LANARKSHIRE,COUNTY,1857-01-01,1921-12-31,55.8775,-4.29406,G12 9,DT
1,16831,COLWYN BAY: COUNCIL OFFICES,CLWYD,COUNTY,1904-01-01,1929-12-31,53.2951,-3.72863,LL29 7,DR DT DW HR HW ST
2,8401,TORQUAY: CARY GREEN,DEVON,COUNTY,1892-01-01,1930-12-31,50.4607,-3.52456,TQ2 5,DR DT DW HR HW ST
3,10654,ROSS-ON-WYE: CRAIG,HEREFORD & WORCESTER,COUNTY,1859-01-01,1943-12-31,51.9162,-2.58309,HR9 7,DR DT DW
4,16789,MOUNT WISE MET OFFICE,DEVON,COUNTY,1946-01-01,1948-12-31,50.3651,-4.16698,PL1 4,DT DW HR


In [9]:
stations_metadata.shape

(1692, 10)

---

## Get Entries w/ Hourly Rainfall and Weather

In [10]:
entries_w_hourly_rainfall_and_weather = stations_metadata[
    stations_metadata['Open datasets available'].map(lambda x: "HW" in x and "HR" in x)
]
entries_w_hourly_rainfall_and_weather

,src_id,Station name,Area,Area type,Station start date,Station end date,Latitude,Longitude,Postcode,Open datasets available
1,16831,COLWYN BAY: COUNCIL OFFICES,CLWYD,COUNTY,1904-01-01,1929-12-31,53.2951,-3.72863,LL29 7,DR DT DW HR HW ST
2,8401,TORQUAY: CARY GREEN,DEVON,COUNTY,1892-01-01,1930-12-31,50.4607,-3.52456,TQ2 5,DR DT DW HR HW ST
6,11169,TENBY: PRIORY,DYFED,COUNTY,1914-01-01,1957-12-31,51.6709,-4.69741,SA70 7,DR DT DW HR HW
35,1312,BATH,AVON,COUNTY,1899-01-01,1963-03-31,51.3857,-2.35355,BA2 6,DR DT DW HR HW ST
42,783,TANGMERE,WEST SUSSEX,COUNTY,1944-01-01,1963-12-31,50.8494,-0.70576,PO20 6,HR HW
...,...,...,...,...,...,...,...,...,...,...
1680,1530,ARMAGH,ARMAGH,COUNTY,1836-01-01,2022-01-01,54.3523,-6.64866,BT61 7,DR DT DW HR HW MW ST
1685,10268,WINCHCOMBE: SUDELEY CASTLE,GLOUCESTERSHIRE,COUNTY,1922-01-01,2022-01-01,51.9475,-1.95271,GL54 5,DR DT DW HR HW
1688,56486,MONA,ISLE OF ANGLESEY,COUNTY,2009-10-14,2022-01-01,53.2603,-4.37482,LL77 7,DT DW HR HW MW ST
1690,1543,CASTLEDERG,TYRONE,COUNTY,1963-01-01,2022-01-01,54.7070,-7.57668,BT81 7,DR DT DW HR HW MW RA ST


In [11]:
stations_metadata = entries_w_hourly_rainfall_and_weather

---

## Date Helper Functions

In [12]:
def get_date_from_race_data(date: str) -> datetime:
    # the strip here is a hack until we can fix elsewhere, similarly the prepend with 0
    date = date.rstrip(' 00:00')
    date = '0' + date if date[1] == '/' else date
    return datetime.strptime(date, '%y/%m/%d')

In [13]:
def get_date_from_stations_metadata(date: str) -> datetime:
    return datetime.strptime(date, '%Y-%m-%d')

In [14]:
def get_open_stations(df: pd.core.frame.DataFrame, race_date: datetime) -> List[bool]:
    return [
        (get_date_from_stations_metadata(row['Station start date']) < race_date) and \
        (get_date_from_stations_metadata(row['Station end date']) > race_date)
        for _, row
        in df.iterrows()
    ]

In [15]:
def station_is_open(row: pd.core.frame.DataFrame, race_date: datetime) -> bool:
    return (get_date_from_stations_metadata(row['Station start date']) < race_date) and \
           (get_date_from_stations_metadata(row['Station end date']) > race_date)

---

## Distance Helper Functions

**Note**: The precise distance between two points specified by `(latitude, longitude)` coordinates cannot be computed by the Euclidean distance formula, as this would instead calculate the distance if you drilled a wire into Earth (and ignored the curvature of Earth). However, for points which are near to each other (and not across the world), this is a fair approximation. Additionally, the precise formula, give here [https://stackoverflow.com/questions/28994289/calculate-euclidean-distance-with-google-maps-coordinates#:~:text=You%20can%2C%20but%20not%20by,from%20a%20degree%20of%20latitude.](https://stackoverflow.com/questions/28994289/calculate-euclidean-distance-with-google-maps-coordinates#:~:text=You%20can%2C%20but%20not%20by,from%20a%20degree%20of%20latitude.) (among other sources) involves a calculation with cosine, which is more computationally expensive. Furthermore, we find it unlikely that there are several stations approximately equidistant from a given track. For all these reasons, we will just use the Euclidean distance for this decision.

In [16]:
def get_distance_to_station(df: pd.core.frame.DataFrame, track_lat: float, track_lng: float) -> float:
    champion_station_name = ''
    champion_area = ''
    champion_distance = np.inf
    
    for _, row in df.iterrows():
        dist = (row['Latitude'] - track_lat) ** 2 + (row['Longitude'] - track_lng) ** 2
        
        if dist < champion_distance:
            champion_station_name = row['Station name']
            champion_area = row['Area']
            champion_distance = dist
            
    return (champion_station_name, champion_area)

---

## Make `rid` to Station Dict (Attempt 1)

As far as I can tell, this should work fine but it is too slow for our purposes.

In [17]:
# d = {}

# for idx, row in tqdm(races_aticnmg.iterrows()):
    
#     race_date = get_date_from_race_data(row['date'])
    
#     # first, filter all stations that are closed when this race occurs
#     flter_lst = stations_metadata[get_open_stations(stations_metadata, race_date)]
    
#     if len(flter_lst) == 0:
#         print(idx, row)
#         continue
    
#     # second, find the station with the nearest distance
#     location = COURSE_AND_COUNTRY_TO_LOCATION[(row['course'], row['countryCode'])]
#     station_name, area = get_distance_to_station(flter_lst, location['lat'], location['lng'])
    
#     d[row['rid']] = (station_name, area)

---

## Make `rid` to Station Dict (Attempt 2)

A second attempt instead precomputes lengths to different stations for easy lookup. That is, we will make a dictionary from each `(course, countryCode)` pair to an ordered list of `(Station name, Area)` pairs and associated distances, where distances are in increasing order.

In [18]:
COURSE_AND_COUNTRY_TO_LOCATION_GB = {}

for key, val in COURSE_AND_COUNTRY_TO_LOCATION.items():
    if key[1] == 'GB':
        COURSE_AND_COUNTRY_TO_LOCATION_GB[key] = val

In [19]:
COURSE_AND_COUNTRY_TO_STATION = {}

for key, val in tqdm(COURSE_AND_COUNTRY_TO_LOCATION_GB.items()):
    distances = []
    for _, row in stations_metadata.iterrows():
        distance = (row['Latitude'] - val['lat']) ** 2 + (row['Longitude'] - val['lng']) ** 2
        distances.append(((row['Station name'], row['Area']), distance))
    distances = sorted(distances, key=lambda x: x[1])
    COURSE_AND_COUNTRY_TO_STATION[key] = distances

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:01<00:00, 45.59it/s]


Now, for each race, we can find its corresponding sorted list of distances and find the first in the list that is open and collecting data over an interval which contains the race date.

In [20]:
d = {}

for idx, row in tqdm(races_aticnmg.iterrows()):
    
    race_date = get_date_from_race_data(row['date'])
    
    # get sorted list of stations
    lst = COURSE_AND_COUNTRY_TO_STATION[(row['course'], row['countryCode'])]
    
    # elt is ((station name, area), dist)
    for elt in lst:
        station_row = stations_metadata[stations_metadata['Station name'] == elt[0][0]].iloc[0]
        if station_is_open(station_row, race_date):
            d[row['rid']] = elt[0][0], elt[0][1], race_date.year
            break

123972it [01:28, 1403.07it/s]


In [21]:
len(d)

123972

In [28]:
smple = races_aticnmg.sample(5)

In [29]:
smple

,rid,course,time,date,hurdles,prizes,winningTime,metric,countryCode,ncond,class
91759,111461,Goodwood,06:40,15/08/28,NaN,"[3234.5, 962.5, 481.0, 240.5]",215.76,3218.0,GB,5,5
43168,241443,Wolverhampton (AW),08:00,06/09/02,NaN,"[2388.75, 705.25, 352.8]",159.89,2413.0,GB,14,6
13530,99476,Windsor,02:15,19/06/29,NaN,"[12450.0, 3728.0, 1864.0, 932.0, 466.0]",144.45,2312.5,GB,2,2
99807,75497,Carlisle,04:30,17/06/28,NaN,"[6469.0, 1925.0, 962.0, 481.0]",91.45,1407.0,GB,10,4
9182,157689,Doncaster,03:35,18/09/15,NaN,"[421355.3, 159745.0, 79946.8, 39824.8, 19986.7...",183.34,2915.5,GB,1,1


In [30]:
[(rid, d[rid]) for rid in smple['rid']]

[(111461, ('THORNEY ISLAND', 'WEST SUSSEX', 2015)),
 (241443, ('WINTERBOURNE: UNIVERSITY OF BIRMINGHAM', 'WEST MIDLANDS', 2006)),
 (99476, ('HEATHROW', 'GREATER LONDON', 2019)),
 (75497, ('CARLISLE', 'CUMBRIA', 2017)),
 (157689, ('GRINGLEY-ON-THE-HILL', 'NOTTINGHAMSHIRE', 2018))]

In [31]:
stations_metadata[stations_metadata['Station name'].isin([d[rid][0] for rid in smple['rid']])]

,src_id,Station name,Area,Area type,Station start date,Station end date,Latitude,Longitude,Postcode,Open datasets available
1138,586,WINTERBOURNE: UNIVERSITY OF BIRMINGHAM,WEST MIDLANDS,COUNTY,1980-01-01,2012-05-15,52.4541,-1.92295,B15 2,DR DT DW HR HW RA ST
1490,779,THORNEY ISLAND,WEST SUSSEX,COUNTY,1941-01-01,2022-01-01,50.8142,-0.92098,PO10 8,DR DT DW HR HW MW RA ST
1582,1070,CARLISLE,CUMBRIA,COUNTY,1961-01-01,2022-01-01,54.9342,-2.96223,CA6 4,DR DT DW HR HW MW RA ST
1634,708,HEATHROW,GREATER LONDON,COUNTY,1947-01-01,2022-01-01,51.4787,-0.44904,TW6 2,DR DT DW HR HW MW RA ST
1659,19204,GRINGLEY-ON-THE-HILL,NOTTINGHAMSHIRE,COUNTY,1996-12-14,2022-01-01,53.4059,-0.88265,DN10 4,DR DT DW HR HW MW RA ST


These are correct by inspection.

---

## Write to File in `utils`

In [32]:
s = f"RID_TO_STATION = {d}"
s[:100]

"RID_TO_STATION = {267255: ('GRINGLEY-ON-THE-HILL', 'NOTTINGHAMSHIRE', 1997), 297570: ('GRINGLEY-ON-T"

In [33]:
with open(f"{BASE_DIR}/utils/rid_to_station.py", 'w', encoding='utf-8') as f:
    f.write(s)

---